Instalação da biblioteca icd10-cm.

É uma biblioteca voltada para a Classificação Internacional de Doenças, Décima Revisão (CID-10) é um sistema usado por médicos para classificar e codificar todos os diagnósticos, sintomas e procedimentos para processamento de sinistros.

In [ ]:
!pip install icd10-cm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.4/738.4 kB 5.5 MB/s eta 0:00:00


# Instalação de Bibliotecas 📚


In [ ]:
import requests
import json
import pandas as pd
import icd10

A função getDataFromSIM realiza uma solicitação HTTP GET para uma URL específica usando a biblioteca requests e retorna o conteúdo da resposta em formato JSON.

In [ ]:
def getDataFromSIM(ano, local,abrangencia,pais,lococor,indicador,faixa,racacor):
  url = "https://svs.aids.gov.br/services2/v1/mortalidade/cid10/exportar/localidade"

  querystring = {"ano":ano,"local":local,"abrangencia":abrangencia,"pais":pais,"categoria":"1","estatistica":"1","lococor":lococor,"grupoetario":faixa,"racacor":racacor,"sexo":"0","indicador":indicador,"atestante":"0","espacial":"sus","parcial":"false"}

  payload = ""
  headers = {
      "Accept": "*/*",
      "Accept-Language": "pt-BR,pt;q=0.9",
      "Connection": "keep-alive",
      "Referer": "https://svs.aids.gov.br/daent/centrais-de-conteudos/paineis-de-monitoramento/mortalidade/cid10/",
      "Sec-Fetch-Dest": "empty",
      "Sec-Fetch-Mode": "cors",
      "Sec-Fetch-Site": "same-origin",
      "Sec-GPC": "1",
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
      "sec-ch-ua": '"Brave";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
      "sec-ch-ua-mobile": "?0",
      "sec-ch-ua-platform": '"Ubuntu"'
  }

  response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
  print(response.text)
  return response.json()

In [ ]:
url = "https://svs.aids.gov.br/services2/v1/mortalidade/cid10/filtro/indicador"
payload = ""
headers = {
    "Accept": "*/*",
    "Accept-Language": "pt-BR,pt;q=0.9",
    "Connection": "keep-alive",
    "Referer": "https://svs.aids.gov.br/daent/centrais-de-conteudos/paineis-de-monitoramento/mortalidade/cid10/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Sec-GPC": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Brave";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}

response = requests.request("GET", url, data=payload, headers=headers)

print(response.text)

{"resumo":{"parametro":"indicador","texto":"Indicador"},"filtros":{},"resultados":[{"root":0,"uid":200001,"nome":"Óbitos totais","padrao":true},{"root":200001,"uid":200002,"nome":"(A00-B99) Algumas doenças infecciosas e parasitárias","padrao":false},{"root":200002,"uid":200003,"nome":"(A00-A09) Doenças infecciosas intestinais","padrao":false},{"root":200003,"uid":200004,"nome":"(A00) Cólera","padrao":false},{"root":200004,"uid":200005,"nome":"(A00.0) Cólera devida a Vibrio cholerae 01, biótipo cholerae","padrao":false},{"root":200004,"uid":200006,"nome":"(A00.1) Cólera devida a Vibrio cholerae 01, biótipo El Tor","padrao":false},{"root":200004,"uid":200007,"nome":"(A00.9) Cólera não especificada","padrao":false},{"root":200003,"uid":200008,"nome":"(A01) Febres tifóide e paratifóide","padrao":false},{"root":200008,"uid":200009,"nome":"(A01.0) Febre tifóide","padrao":false},{"root":200008,"uid":200010,"nome":"(A01.1) Febre paratifóide A","padrao":false},{"root":200008,"uid":200011,"nome"

In [ ]:
indicador_df = pd.json_normalize(response.json()['resultados'])

Série de operações para coletar dados do Sistema de Informação de Mortalidade (SIM), com base em informações contidas em um objeto chamado icd10.chapters e os utiliza para fazer chamadas à função getDataFromSIM.

In [ ]:
uids = []
dfs = []
for chapter, range, desc in icd10.chapters:
  temp_uid = indicador_df['uid'][indicador_df['nome'].str.contains(range)]
  print(int(temp_uid))
  temp = getDataFromSIM(2020,1,1,76,0,temp_uid,2000,0)
  temp_df = pd.json_normalize(temp, record_path=['resultados'], meta=[['filtros', 'ano','uid'], ['filtros', 'local','nome'], ['filtros', 'indicador', 'uid'],['filtros', 'indicador', 'nome'],['filtros', 'grupoetario', 'nome'],['filtros', 'racacor', 'nome'],['filtros', 'sexo', 'nome']])
  dfs.append(temp_df)

200002
{"resumo":{"parametro":"localidade","texto":"Localidade","data":"Dezembro de 2023","sigla":"SIM","fonte":"Sistema de Informações sobre Mortalidade"},"filtros":{"ano":{"uid":2020,"nome":"2020"},"local":{"uid":1,"nome":"Óbitos por residência"},"indicador":{"uid":200002,"nome":"(A00-B99) Algumas doenças infecciosas e parasitárias"},"categoria":{"uid":1,"nome":"Notificação de óbitos"},"estatistica":{"uid":1,"nome":"Número de óbitos"},"lococor":{"uid":0,"nome":"Todos"},"atestante":{"uid":0,"nome":"Todos"},"grupoetario":{"uid":2000,"nome":"Todos"},"racacor":{"uid":0,"nome":"Todos"},"sexo":{"uid":0,"nome":"Todos"}},"resultados":[{"abrangencia":{"uid":1,"nome":"País"},"uid":76,"nome":"Brasil","jan":4735.0,"fev":4648.0,"mar":5854.0,"abr":16828.0,"mai":38707.0,"jun":34733.0,"jul":36315.0,"ago":31426.0,"set":23695.0,"out":19053.0,"nov":19057.0,"dez":32236.0,"ano":267287.0},{"abrangencia":{"uid":2,"nome":"Região"},"uid":1,"nome":"Norte","jan":386.0,"fev":390.0,"mar":454.0,"abr":2772.0,"mai"

# Análise Exploratória de Dados  🔍

In [ ]:
mortes_df = pd.concat(dfs)
mortes_df.reset_index(inplace=True)
mortes_df

,index,uid,nome,jan,fev,mar,abr,mai,jun,jul,...,ano,abrangencia.uid,abrangencia.nome,filtros.ano.uid,filtros.local.nome,filtros.indicador.uid,filtros.indicador.nome,filtros.grupoetario.nome,filtros.racacor.nome,filtros.sexo.nome
0,0,76,Brasil,4735.0,4648.0,5854.0,16828.0,38707.0,34733.0,36315.0,...,267287.0,1,País,2020,Óbitos por residência,200002,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,Todos
1,1,1,Norte,386.0,390.0,454.0,2772.0,6028.0,3616.0,2509.0,...,24113.0,2,Região,2020,Óbitos por residência,200002,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,Todos
2,2,11,Rondônia,32.0,24.0,27.0,58.0,210.0,417.0,387.0,...,2212.0,3,Unidade da federação,2020,Óbitos por residência,200002,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,Todos
3,3,12,Acre,13.0,27.0,15.0,40.0,159.0,249.0,163.0,...,1006.0,3,Unidade da federação,2020,Óbitos por residência,200002,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,Todos
4,4,13,Amazonas,101.0,97.0,112.0,1467.0,1844.0,627.0,451.0,...,6972.0,3,Unidade da federação,2020,Óbitos por residência,200002,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,Todos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,28,5,Centro-Oeste,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,Região,2020,Óbitos por residência,214514,(U00-U99) Códigos para propósitos especiais,Todos,Todos,Todos
722,29,50,Mato Grosso do Sul,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,Unidade da federação,2020,Óbitos por residência,214514,(U00-U99) Códigos para propósitos especiais,Todos,Todos,Todos
723,30,51,Mato Grosso,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,Unidade da federação,2020,Óbitos por residência,214514,(U00-U99) Códigos para propósitos especiais,Todos,Todos,Todos
724,31,52,Goiás,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,Unidade da federação,2020,Óbitos por residência,214514,(U00-U99) Códigos para propósitos especiais,Todos,Todos,Todos


In [ ]:
mortes_pais_df = mortes_df[mortes_df['abrangencia.uid'] == 1].sort_values(by = 'ano', ascending=False).reset_index()
mortes_pais_df['ano_percentual'] = mortes_pais_df['ano'].apply(lambda x: round(100*x/mortes_pais_df['ano'].sum(),2))

In [ ]:
colunas_para_remover = ['level_0', 'index', 'uid', 'nome', 'jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez', 'abrangencia.uid', 'abrangencia.nome', 'filtros.local.nome', 'filtros.indicador.uid','filtros.grupoetario.nome']

mortes_pais_df_sem_colunas = mortes_pais_df.drop(columns=colunas_para_remover)



In [ ]:
mortes_pais_df_sem_colunas

,ano,filtros.ano.uid,filtros.indicador.nome,filtros.racacor.nome,filtros.sexo.nome,ano_percentual
0,357741.0,2020,(I00-I99) Doenças do aparelho circulatório,Todos,Todos,22.98
1,267287.0,2020,(A00-B99) Algumas doenças infecciosas e parasi...,Todos,Todos,17.17
2,229300.0,2020,(C00-D48) Neoplasias [tumores],Todos,Todos,14.73
3,148773.0,2020,(J00-J99) Doenças do aparelho respiratório,Todos,Todos,9.56
4,146038.0,2020,(V01-Y98) Causas externas de morbidade e de mo...,Todos,Todos,9.38
5,92749.0,2020,"(E00-E90) Doenças endócrinas, nutricionais e m...",Todos,Todos,5.96
6,90345.0,2020,"(R00-R99) Sintomas, sinais e achados anormais ...",Todos,Todos,5.80
7,66667.0,2020,(K00-K93) Doenças do aparelho digestivo,Todos,Todos,4.28
8,45598.0,2020,(G00-G99) Doenças do sistema nervoso,Todos,Todos,2.93
9,44860.0,2020,(N00-N99) Doenças do aparelho geniturinário,Todos,Todos,2.88


In [ ]:
soma_ano = mortes_pais_df['ano'].sum()
print("morreram", soma_ano," pessoas no ano de 2020 ")



morreram 1556824.0  pessoas no ano de 2020 


In [ ]:

mortes_pais_df_sem_colunas.to_csv('./morte_todos_todos.csv', sep=';', encoding='utf-8', index=False)



Códigos utilizados para cada faixa etária

Infantil: 2001

01 a 04: 2002

05 a 09: 2003

10 a 14: 2004

15 a 19: 2005

20 a 29: 2006

30 a 39: 2007

40 a 49: 2008

50 a 59: 2009

60 a 69: 2010

70 a 79: 2011

80 e +: 2012

Branco/Ignorado: 2999

Todos: 2000

In [ ]:
from google.colab import files

files.download('./morte_todos_todos.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>